## **Laboratorio 5: Parte 1 - Algoritmos genéticos**
### **Actividades: Chameleon y Maze Runner**

**ISIS-3302:** Modelado, Simulación y Optimización

Departamento de Ingeniería de Sistemas y Computación

Universidad de los Andes

### **Integrantes - Grupo 15**

Daniel Felipe Diaz Moreno y Sara Sofía Cárdenas Rodríguez

### 1. Definición del Problema

- **Descripción Detallada**: Explicación del objetivo de camuflar el cubo ajustando su color en escala de grises.
- **Justificación**: Razones por las cuales se utiliza un algoritmo genético para resolver este problema.

### 2. Diseño del Algoritmo Genético

#### 2.1. Especificación de Componentes: Detalles de cada componente del algoritmo (representación genética, función de aptitud, operadores genéticos, etc.).



##### 2.1.1. Representación Genética (Codificación)

- **Genotipo**, **Fenotipo** y **Codificación**: Para cada individuo, se definirá su color dentro del atributo fenotipo, siendo un valor entre 0 y 255.

##### 2.1.2. Población Inicial

- **Tamaño de la Población**: 80 individuos.
- **Inicialización**: Los valores de gris de los individuos se generan aleatoriamente entre 0 y 255.

##### 2.1.3. Función de Aptitud (Fitness Function)

- **Objetivo**: Minimizar la diferencia absoluta entre el nivel de gris del individuo y el nivel de gris del fondo.
- **Función de Aptitud**: \( \text{Aptitud}(i) = -|\text{Gris}_{\text{individuo}} - \text{Gris}_{\text{fondo}}| \)


##### 2.1.4. Selección

- **Criterio de Supervivencia**: Solo el 20% de los individuos con mayor aptitud sobreviven para la siguiente generación.
- **Método de Selección**: Selección elitista, donde se conservan los mejores individuos.

##### 2.1.5. Operadores Genéticos

##### a. Cruce (Crossover)

- **Tipo de Cruce**: Cruce uniforme.
- **Proceso**: Se seleccionan pares de padres y nacen dos nuevos hijos, uno con la suma de los genotipos de sus padres y otro con el promedio. A ambos valores se les aplica el modulo para evitar valores no validos

##### b. Mutación

- **Probabilidad de Mutación**:  1%.
- **Proceso**: Se añade o resta un valor aleatorio entre -15 y 15 al color del individuo

##### 2.1.6. Criterio de Parada

- **Condiciones**:
  - Alcanzar un nivel de aptitud satisfactorio (la diferencia debe ser de 0).
  - Llegar a un número máximo de generaciones.

#### 2.2. Diagrama de Flujo: Visualización de los pasos del algoritmo para facilitar la comprensión del proceso.
![Diagrama de flujo](flujo.png)

### 3. Implementación

- **Código Fuente**: Implementación del algoritmo en un lenguaje de programación adecuado (por ejemplo, Python, Java).
- **Comentarios y Documentación**: Explicaciones dentro del código para clarificar las funciones y procedimientos utilizados.



In [14]:
import random

class IndividuoCamuflaje:
    def __init__(self, camuflaje=None):
        self.genotipo = camuflaje or random.randint(0, 255)
        self.fitness = 0

class AlgoritmoGeneticoCamuflaje:
    def __init__(self, poblacion, generaciones, tasaCruce, tasaMutacion, elitismo, fondo):
        self.tamañoPoblacion = poblacion
        self.generaciones = generaciones
        self.tasaCruce = tasaCruce
        self.tasaMutacion = tasaMutacion
        self.elitismo = int(elitismo * poblacion)
        self.poblacion = self.inicializarPoblacion(self.tamañoPoblacion)
        self.generacion = 0
        self.fondo = fondo

    def inicializarPoblacion(self, poblacion):
        return [IndividuoCamuflaje() for _ in range(poblacion)]

    def evaluarPoblacion(self):
        for individuo in self.poblacion:
            individuo.fitness = self.evaluarFitness(individuo)

    def evaluarFitness(self, individuo):
        return -abs(individuo.genotipo - self.fondo)

    def seleccionarPadres(self):
        return random.choices(self.poblacion, k=2)

    def cruzar(self, padre1, padre2):
        hijo1 = IndividuoCamuflaje((padre1.genotipo + padre2.genotipo) % 256)
        hijo2 = IndividuoCamuflaje((padre1.genotipo + abs(padre1.genotipo - padre2.genotipo)//2 ) % 256)
        return hijo1, hijo2

    def mutar(self, individuo):
        individuo.genotipo = (individuo.genotipo + random.randint(-15, 15)) % 256

    def condicionDeTerminacion(self):
        return abs(self.mejorIndividuo().fitness) == 0

    def aplicarElitismo(self, poblacion, nuevaPoblacion, elitismo):
        poblacion.sort(key=lambda individuo: individuo.fitness, reverse=True)
        nuevaPoblacion.sort(key=lambda individuo: individuo.fitness, reverse=True)
        return poblacion[:elitismo] + nuevaPoblacion[:self.tamañoPoblacion - elitismo]

    def mejorIndividuo(self):
        return max(self.poblacion, key=lambda individuo: individuo.fitness)
    
    def obtenerMetricas(self):
        mejor = self.mejorIndividuo()
        return {
            'generacion': self.generacion,
            'fitness': mejor.fitness,
            'genotipo': mejor.genotipo,
            'fondo': self.fondo
        }

    def ejecutar(self):
        self.evaluarPoblacion()
        while self.generacion < self.generaciones and not self.condicionDeTerminacion():
            nuevaPoblacion = []
            for _ in range(self.tamañoPoblacion - self.elitismo):
                padre1, padre2 = self.seleccionarPadres()
                if random.random() < self.tasaCruce:
                    hijo1, hijo2 = self.cruzar(padre1, padre2)
                else:
                    hijo1, hijo2 = padre1, padre2
                nuevaPoblacion.extend([hijo1, hijo2])
            for individuo in nuevaPoblacion:
                if random.random() < self.tasaMutacion:
                    self.mutar(individuo)

            self.poblacion = self.aplicarElitismo(self.poblacion, nuevaPoblacion, self.elitismo)

            self.evaluarPoblacion()

            self.generacion += 1
        return self.mejorIndividuo()



### 4. Pruebas y Resultados

- **Datos de Prueba**: Información sobre el fondo (nivel de gris) y parámetros utilizados.
- **Ejecución del Algoritmo**: Descripción de cómo se ejecutó el algoritmo, incluyendo el número de generaciones y tiempos de ejecución.
- **Resultados Obtenidos**: Presentación de los niveles de gris de los individuos a lo largo de las generaciones y cómo se acercan al nivel de gris del fondo.



In [47]:
tamañoPoblacion = 80
generaciones = 100
probabilidadCruce = 0.8
elitismo = 0.2
probabilidadMutacion = 0.01
fondo = 128

algoritmoGeneticoCamuflaje = AlgoritmoGeneticoCamuflaje(tamañoPoblacion, generaciones, probabilidadCruce, probabilidadMutacion, elitismo, fondo)
algoritmoGeneticoCamuflaje.ejecutar()
print(algoritmoGeneticoCamuflaje.obtenerMetricas())

{'generacion': 4, 'fitness': 0, 'genotipo': 128, 'fondo': 128}


Es importante resaltar que al tener un tamaño de población de 80 individuos, es bastante probable que el algoritmo encuentre la solución optima sin necesidad de iterar, de hecho, esta probabilidad corresponde a cerca del 27% 

La probabilidad de adivinar correctamente un número entre 0 y 255 en un solo intento es:

$$
P(\text{adivinar}) = \frac{1}{256}
$$

La probabilidad de no adivinar en un intento es:

$$
P(\text{no adivinar}) = 1 - P(\text{adivinar}) = 1 - \frac{1}{256} = \frac{255}{256}
$$

La probabilidad de no adivinar en 80 intentos es:

$$
P(\text{no adivinar en 80 intentos}) = \left( \frac{255}{256} \right)^{80}
$$

Finalmente, la probabilidad de adivinar al menos una vez en 80 intentos es:

$$
P(\text{adivinar al menos una vez}) = 1 - P(\text{no adivinar en 80 intentos}) = 1 - \left( \frac{255}{256} \right)^{80}
$$

Al calcular esto, obtenemos una probabilidad de aproximadamente **0.269** o **26.9%**.

### 5. Caso de Estudio: El color de fondo va a cambiar a lo largo de las generaciones
- **Descripción del Problema**: Se desea camuflar un cubo en un entorno donde el color de fondo cambia a lo largo del tiempo.
- **Modificación del Algoritmo**: Se debe adaptar el algoritmo genético para manejar la variabilidad del color de fondo y ajustar el cubo en consecuencia.
- **Consideraciones Adicionales**: Se debe evaluar cómo la variabilidad afecta el desempeño del algoritmo y si se requieren ajustes en los parámetros.



Se aumenta la cantidad de individuos de 80 a 100 para mejorar la precisión del algoritmo genético, disminuyendo la cantidad de generaciones y la diferencia absoluta con respecto al color actual

In [ ]:
class AlgoritmoGeneticoCamuflajeCambiante(AlgoritmoGeneticoCamuflaje):
    def __init__(self, poblacion, generaciones, tasaCruce, tasaMutacion, elitismo, fondo):
        super().__init__(poblacion, generaciones, tasaCruce, tasaMutacion, elitismo, fondo)
        
    def cambiarFondo(self):
        self.fondo = (self.fondo + random.randint(-25, 25)) % 256

    def ejecutar(self):
        self.evaluarPoblacion()
        while self.generacion < self.generaciones and not self.condicionDeTerminacion():
            nuevaPoblacion = []
            for _ in range(self.tamañoPoblacion - self.elitismo):
                padre1, padre2 = self.seleccionarPadres()
                if random.random() < self.tasaCruce:
                    hijo1, hijo2 = self.cruzar(padre1, padre2)
                else:
                    hijo1, hijo2 = padre1, padre2
                nuevaPoblacion.extend([hijo1, hijo2])
            for individuo in nuevaPoblacion:
                if random.random() < self.tasaMutacion:
                    self.mutar(individuo)

            self.poblacion = self.aplicarElitismo(self.poblacion, nuevaPoblacion, self.elitismo)

            self.cambiarFondo()
            self.evaluarPoblacion()
            self.generacion += 1
        return self.mejorIndividuo()


In [56]:
tamañoPoblacion = 100
generaciones = 100
probabilidadCruce = 0.8
elitismo = 0.2
probabilidadMutacion = 0.01
fondo = 128

algoritmoGeneticoCamuflaje = AlgoritmoGeneticoCamuflajeCambiante(tamañoPoblacion, generaciones, probabilidadCruce, probabilidadMutacion, elitismo, fondo)
algoritmoGeneticoCamuflaje.ejecutar()
print(algoritmoGeneticoCamuflaje.obtenerMetricas())

{'generacion': 2, 'fitness': 0, 'genotipo': 144, 'fondo': 144}


### 6. Análisis de Resultados

- **Gráficas y Tablas**: 
  - Visualización del progreso del algoritmo, como gráficas de aptitud promedio y máxima por generación.
  - Visualización de el camuflaje obtenido para esa generación.
- **Interpretación**: Análisis de cómo el algoritmo logró mejorar la aptitud de los individuos y si se alcanzó el objetivo de camuflaje.
- **Discusión de Desempeño**: Evaluación de la eficacia del algoritmo y posibles mejoras.



### 7. Conclusiones

- **Resumen de Hallazgos**: Síntesis de los resultados y su relevancia.
- **Aprendizajes**: Lecciones obtenidas del desarrollo y ejecución del algoritmo.
- **Recomendaciones**: Sugerencias para futuras implementaciones o ajustes al algoritmo.
